
# Prediksi Hilal
---

In [1]:
# _jengkolrebus
# Mei, 2020
# Curup, Bengkulu

Program ini menggunakan Python Package: [Skyfield](https://rhodesmill.org/skyfield/).
Program ini dibuat dengan ketentuan sebagai berikut:
- Ephemeris:
    > Ephemeris yang digunakan adalah de421.bsp
- Definisi Matahari Terbenam:
    > Skyfield menggunakan definisi resmi matahari terbit dan terbenam dari United States Naval Observatory, yang mendefinisikan matahari terbenam ketika pusat matahari adalah <b>0,8333</b> derajat di bawah Horizon, untuk memperhitungkan rata-rata jari-jari Matahari dan untuk rata-rata pembiasan atmosfer di horizon.
___
- Flowchart:
<br>
<img src="flowchart.png">
<br>
- Input Lokasi berupa Latitude dan Longitude dalam bentuk <em>Decimal Degree</em> dengan format String / text.
 > Contoh : 
    Latitude: "7.83305556 S"
    Longitude: "110.38305556 E"
    Perhatikan Notasi "S" dan "E" bergantung pada lokasi.
    
- Input waktu berupa tanggal rentang waktu hilal yang ingin dicari.
 > Contoh dari tanggal 1 Januari 2020, sampai 1 Januari 2021
___


In [2]:
from skyfield.api import load, Topos
from skyfield.units import Angle
from skyfield import almanac
from datetime import datetime, timedelta
from pytz import timezone
from ipywidgets import widgets, interact, interactive
from IPython.display import display, HTML
import pandas as pd
import calendar

In [3]:
jkt = timezone('Asia/Jakarta')
ts = load.timescale()
e = load('de421.bsp')

In [4]:
# Membuat variable-variable global
class var:
    topo=None
    loc=None
    MOON_RADIUS = 1737.1 # km
    SUN_RADIUS = 696340 # km
    konjungsi = []
    sunset = []
    alt_bulan = []
    az_bulan = []
    jarak_bulan = []
    alt_matahari = []
    az_matahari = []
    jarak_matahari = []
    elong = []
    ir = []
    moonset = []
    usia_bulan = []

### Kriteria Hilal
- Imkanur Rukyat
    > - Pada saat matahari terbenam, <b>ketinggian bulan minimal 2$^\circ$</b>, <b>Elongasi minimal 3$^\circ$</b>.
    > - Pada saat Bulan terbenam, usia bulan minimal <b>8 Jam</b>, dihitung sejak ijtima' (konjungsi)
    
___

In [5]:
# Metode untuk membandingkan dengan Imkan Rukyat
def imkanRukyat():
    alt = var.alt_bulan
    elong = var.elong
    usia_bulan = var.usia_bulan
    for i, j, k in zip(alt, elong, usia_bulan):
        if(i.degrees >= 2 and j.degrees >= 3 and (timedelta.total_seconds(k)/3600) > 8):
            var.ir.append(u'\u2714')
        else:
            var.ir.append(u'\u2718')

In [6]:
# Menghapus data yang tersimpan dalam variable, agar tidak menumpuk ketika pengaturan lokasi/waktu diubah
def clearVar():
    var.konjungsi.clear()
    var.sunset.clear()
    var.alt_bulan.clear()
    var.az_bulan.clear()
    var.jarak_bulan.clear()
    var.alt_matahari.clear()
    var.az_matahari.clear()
    var.jarak_matahari.clear()
    var.elong.clear()
    var.ir.clear()
    var.usia_bulan.clear()
    var.moonset.clear()

# Metode untuk mendapatkan posisi benda langit
def posisiObyek(obj, t):
    obj = e[obj]
    astrometric = var.loc.at(t).observe(obj)
    alt, az, d = astrometric.apparent().altaz()
    return astrometric, alt, az, d.km

def usiaBulan():
    for i, j in zip(var.konjungsi, var.moonset):
        t0 = j
        t1 = i
        dt = t0-t1
#         (h, m, s) = str(dt).split(':')
#         result = (int(h)) + (int(m) / 60) + (float(s)/3600)
#         dt = timedelta.total_seconds(dt)/3600
        var.usia_bulan.append(dt)

def moonset(t):
    konj = t
    t = t.utc
    t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
    t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
    f = almanac.risings_and_settings(e, e['moon'], var.topo)
    t, y = almanac.find_discrete(t0, t1, f)
    
    for ti, yi in zip(t, y):
        if(yi==False):
            var.moonset.append(ti.astimezone(jkt))
        else:
            pass
# Metode untuk mendapatkan waktu Sunset
def sunset(t):
    t_konj = t
    t = t.utc
    t0 = ts.utc(t[0], t[1], t[2], t[3], t[4], t[5])
    t1 = ts.utc(t[0], t[1], t[2]+1, t[3], t[4], t[5])
    f = almanac.sunrise_sunset(e, var.topo)
    t, y = almanac.find_discrete(t0, t1, f)
    
    for ti, yi in zip(t, y):
        if(yi==False):
            var.sunset.append(ti.astimezone(jkt))
            
            # Mendapatkan posisi Bulan
            bulan = posisiObyek('moon', ti)
            var.alt_bulan.append(bulan[1])
            var.az_bulan.append(bulan[2])
            var.jarak_bulan.append(bulan[3])
            
            # Mendapatkan posisi Matahari
            matahari = posisiObyek('sun', ti)
            var.alt_matahari.append(matahari[1])
            var.az_matahari.append(matahari[2])
            var.jarak_matahari.append(matahari[3])
            
            # Mendapatkan Elongasi
            elong = bulan[0].separation_from(matahari[0])
            var.elong.append(elong)
            
            
            
            moonset(ti)
        else:
            pass

# Metode untuk mendapatkan waktu konjungsi
def konjungsi(t0, t1):
    t0 = ts.utc(t0)
    t1 = ts.utc(t1)
    f = almanac.oppositions_conjunctions(e, e['moon'])
    t, y = almanac.find_discrete(t0, t1, f)
    
    for ti, yi in zip(t, y):
        if(yi==1):
            var.konjungsi.append(ti.astimezone(jkt))
            
            # Mendapatkan Waktu sunset setelah konjungsi
            sunset(ti)
        else:
            pass


In [7]:
# Metode untuk mendapatkan posisi toposentrik
def find(lat, long, t0, t1):
    clearVar()
    
    var.topo = Topos(lat, long)
    print(var.topo)
    var.loc = e['earth']+var.topo
    
    # Mendapatkan waktu konjungsi
    konjungsi(t0, t1)
    
    usiaBulan()
    imkanRukyat()
    # Menampilkan hasil dalam bentuk tabel dataframe
    tabel = list(zip(var.konjungsi, var.sunset,
                     var.alt_bulan, var.az_bulan, 
                     var.alt_matahari, var.az_matahari, 
                    var.elong, var.usia_bulan, var.ir))
    df = pd.DataFrame(tabel, columns=['Waktu Konjungsi (UTC+07)', 'Waktu Hilal (UTC+07)', 
                                      'Altitude Bulan', 'Azimuth Bulan', 
                                      'Altitude Matahari', 'Azimuth Matahari', 
                                     'Elongasi', 'Usia Bulan (jam)', 'Imkan Rukyat'])
    df.index+=1
    display(df)

In [8]:
hari_ini = datetime.now().date()
days_in_month = calendar.monthrange(hari_ini.year, hari_ini.month)[1]
bulan_depan = hari_ini + timedelta(days_in_month)

dari = widgets.DatePicker(value=hari_ini,
    description='Dari:',
    disabled=False
)
sampai = widgets.DatePicker(value=bulan_depan,
    description='Sampai:',
    disabled=False
)

lat = widgets.Text(value='7.83305556 S', description='Latitude:')
long = widgets.Text(value='110.38305556 E', description='Longitude:')

interactive(find, lat=lat, long=long, t0=dari, t1=sampai)


interactive(children=(Text(value='7.83305556 S', description='Latitude:'), Text(value='110.38305556 E', descri…